In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version



In [2]:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

In [3]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf spark-3.2.3-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [4]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-12-29 18:18:22--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.37MB/s    in 0.2s    

2022-12-29 18:18:23 (5.37 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

### Create DataFrames to match tables

In [7]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
review_df = df.dropna()
review_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [8]:
# Create the customers_table DataFrame
customers_df = review_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)","customer_count")
#show top twenty rows
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   36427874|             1|
|   17687096|            10|
|   43798350|             4|
|   49387236|             1|
|   24321836|             2|
|   42543176|             1|
|   16298288|             4|
|   12333316|             1|
|   25790304|             1|
|   13786319|             1|
|   24796052|             1|
|   53049917|             4|
|   30478898|             1|
|   20829560|             3|
|   29699080|             3|
|    8681732|             3|
|   34220092|             2|
|   13727857|            14|
|   13480593|             3|
|   49935991|             1|
+-----------+--------------+
only showing top 20 rows



In [9]:
# Create the products_table DataFrame and drop duplicates. 
products_df = review_df.select(["product_id","product_title"]).drop_duplicates()
#show top twenty rows
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YKA8PX0|        The Basement|
|B004R10N6O|       Morning Glory|
|B006MYGL8S|   Deadwood Season 1|
|B008Y2X78S|Boardwalk Empire ...|
|B003D8XCOC|Eating Out 2: Slo...|
|B01489KEKY|Some Kind of Beau...|
|B00E3WN0TY|   Banshee: Season 1|
|B00DYQNC3E|Masterpiece: Eliz...|
|B00QROEQ7G|             Amnesia|
|B00QWUK1PQ|     Free the Nipple|
|B00ZETHCTE|Louis C.K.: Oh My...|
|B00MQOXI8Y|   The Expendables 3|
|B003E4AXJC|Best of NOVA Volu...|
|B00H6PG6J6|Lee Daniels' The ...|
|B00NFDFCOI|  Words and Pictures|
|B002WXLJ58|The Lord Of The R...|
|B00KGJNQZQ|Robin Williams: W...|
|B005LLSZNM|Sons Of Anarchy S...|
|B0030JBBYW| The Point of Origin|
|B0064J6RU8|Atlas Shrugged Pa...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = review_df.select(["review_id","customer_id","product_id","product_parent","review_date"]).withColumn("review_date", to_date("review_date", 'yyyy-MM-dd').alias("review_date"))
#show top twenty rows
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3FU16928EP5TC|   12190288|B00AYB1482|     668895143| 2015-08-31|
|R1IZHHS1MH3AQ4|   30549954|B00KQD28OM|     246219280| 2015-08-31|
| R52R85WC6TIAH|   52895410|B01489L5LQ|     534732318| 2015-08-31|
| R7HOOYTVIB0DS|   27072354|B008LOVIIK|     239012694| 2015-08-31|
|R1XQ2N5CDOZGNX|   26939022|B0094LZMT0|     535858974| 2015-08-31|
|R1HCST57W334KN|    4772040|B0112OSOQE|      38517795| 2015-08-31|
|R32BUTYQS1ZJBQ|   12910040|B000NPE5SA|     373323715| 2015-08-31|
|  RH4SXPL4L9QU|   38805573|B00XWV4QXG|     633842417| 2015-08-31|
|R37INWIQA5YW8N|   37100714|B00X8UKOUK|     666093513| 2015-08-31|
|R18GSVAAS3N8GX|   41234409|B00OOKXTFU|     801680808| 2015-08-31|
|R2QCG0ZZOHYUOX|   24698954|B00URDJ600|     174981485| 2015-08-31|
|R2B1OCYMUVMS5C|   11956809|B009JUZ5I4|     654293980| 2015-08

In [12]:
# Create the vine_table. DataFrame
vine_df = review_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FU16928EP5TC|          5|            0|          0|   N|                Y|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|                Y|
| R52R85WC6TIAH|          4|           17|         18|   N|                Y|
| R7HOOYTVIB0DS|          5|            0|          0|   N|                Y|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|                Y|
|R1HCST57W334KN|          5|            5|          6|   N|                Y|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|                Y|
|  RH4SXPL4L9QU|          3|            0|          0|   N|                Y|
|R37INWIQA5YW8N|          2|            0|          0|   N|                Y|
|R18GSVAAS3N8GX|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [18]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://digitalvideodb.cjfsykz40lex.us-east-1.rds.amazonaws.com:5432/"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [19]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [21]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [22]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [25]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
# to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.
total_votes_df = vine_df.filter("total_votes>=20")
total_votes_df.orderBy("total_votes", ascending=False).show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2I167RDKRWB5X|          1|         3248|       3974|   N|                Y|
|R1IGU4Z8JVEUVS|          5|         1484|       1638|   N|                N|
|R3D37ASD654872|          5|         1498|       1568|   N|                N|
|R3A65N9EY9K7AB|          5|         1280|       1412|   N|                Y|
|R2Z0G59RK0LYAV|          5|         1338|       1394|   N|                Y|
| RJ5H1YIV6IKFR|          1|          889|       1259|   N|                Y|
| R8E1XQI5SO12T|          5|          794|       1200|   N|                N|
|R3KKH2UILRM66B|          3|         1025|       1127|   N|                N|
|R1PYS6H474P68P|          1|          790|       1117|   N|                N|
|R1IFBGNE1ROQAP|          5|          901|       1026|   N|     

In [26]:
# Filter the total_votes_df and create a new DF to retrieve all the rows where the number of helpful votes
# Divided by total votes is equal to or grater than 50%
helpful_votes_df = total_votes_df.filter("helpful_votes/total_votes>=0.5")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|     

In [32]:
#Filter the helpful_votes_df and create a new DataFrame or table that retrieves all the rows where
# a review was written as part of the Vine program (paid), vine == 'Y'
paid_df = helpful_votes_df.filter("vine == 'Y'")
paid_df.show(10)

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [33]:
# filter the helpful_votes_df and create a new DataFrame or table that retrieves all the rows where
# a review was written as part of the Vine program (unpaid), vine == 'N'

unpaid_df = helpful_votes_df.filter("vine == 'N' ")
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R34CAWTWR5O2VN|          1|           18|         20|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|     

In [37]:
# Determine the total number of reviews, the amount of 5-star reviews, and the percentage of 5-star reviews for the unpaid DF
paid_total_reviews = paid_df.count()
paid_five_star_reviews = paid_df.filter("star_rating==5").count()
#paid_five_star_percentage = paid_five_star_reviews/paid_total_reviews*100
unpaid_df = helpful_votes_df.filter("vine=='N'")
unpaid_total_reviews = unpaid_df.count()
unpaid_five_star_reviews = unpaid_df.filter("star_rating==5").count()
unpaid_five_star_percentage = unpaid_five_star_reviews/unpaid_total_reviews*100
print(f"Total number of unpaid reviews: {unpaid_total_reviews}")
print(f"Total number of unpaid 5-star reviews: {unpaid_five_star_reviews}")
print(f"Percentage of unpaid 5-star reviews: {unpaid_five_star_percentage}")
print(f"Total number of paid reviews: {paid_total_reviews}")
print(f"Total number of paid 5-star reviews: {paid_five_star_reviews}")
#print(f"Percentage of paid 5-star reviews: {paid_five_star_percentage}")

The total number of reviews for unpaid vines were: 11362
The total number of 5 star reviews for unpaid vines were: 4670
The total percent of 5 star reviews for unpaid vines were: 41.10191867628939
